In [23]:
import pandas as pd
import requests
import time
import sys
from Bio import SeqIO
import logomaker
from Bio import AlignIO
from pymsaviz import MsaViz
import plotly.express as px

In [21]:
#requests function
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs)
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

#check the status of the alignment job
def status_check():
    if job_status.text == 'RUNNING' or job_status.text == 'QUEUED':
        print(job_status.text)
        return True
    else:
        print('FINISHED')
        return False
    
#show the alignment once job has completed
def show_alignment():
    query_alignment = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/aln-clustal_num")
    print(query_alignment.text)
    
#turn MSA file into fasta file using BioPython
def convert_clustal_to_fasta(input_file, output_file):
    #parse the CLUSTAL file and read the alignment
    clustal_alignment = AlignIO.read(input_file, "clustal")

    #write the alignment in FASTA format
    with open(output_file, "w") as fasta_file:
        AlignIO.write(clustal_alignment, fasta_file, "fasta")
        
def fasta_to_df(fasta_file):
    """
    Converts sequences in a .fasta file to a dataframe
    where each column is a position in the sequence alignment
    and each row is a sequence.
    """
    # Read sequences from fasta
    sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
    
    # Ensure that all sequences are of the same length
    assert len(set([len(seq) for seq in sequences])) == 1, "All sequences must be of the same length!"
    
    # Convert list of sequences to dataframe
    df = pd.DataFrame([list(seq) for seq in sequences])
    
    return df

In [3]:
data = pd.read_csv('mpnn_results.csv')
data['extracted_sequence'] = data['seq'].apply(lambda x: x.split('/')[-1] if '/' in x else None)
print(data['extracted_sequence'])

0     TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYVLAKIINGKTE...
1     TRECIYYNANWELERTNQSGLERCEGEQDKRLHCFRAYKIINGKPV...
2     TRECIYYNANWELERTNQSGLERCEGEQDKRLHCMVLAKLENGEWK...
3     TRECIYYNANWELERTNQSGLERCEGEQDKRLHCSVLAKLENGKWE...
4     TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYRAWKRENGKDT...
                            ...                        
59    TRECIYYNANWELERTNQSGLERCEGEQDKRLHCLRAAEIKDGKTT...
60    TRECIYYNANWELERTNQSGLERCEGEQDKRLHCIRLSKLENGKVT...
61    TRECIYYNANWELERTNQSGLERCEGEQDKRLHCVLLVRNKNGEWT...
62    TRECIYYNANWELERTNQSGLERCEGEQDKRLHCTRLVRYENGKWT...
63    TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYRLVKYENGVWT...
Name: extracted_sequence, Length: 64, dtype: object


In [4]:
with open('11_typeII_interface.fasta', 'w') as fasta_file:
    for idx, row in data.iterrows():
        fasta_file.write('>sequence_{}\n{}\n'.format(idx + 1, row['extracted_sequence']))

In [5]:
with open('11_typeII_interface.fasta', 'r') as file:
    content = file.read()
print(content)

>sequence_1
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYVLAKIINGKTEILYRGCVEYNDECYDRQECVSTEKDSKYLFCCCEGNFCNERFTHLPEPG
>sequence_2
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCFRAYKIINGKPVVVYSGCVEYNDECYDRQECVSTEKNARIKFCCCEGNFCNERFTHLPEPG
>sequence_3
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCMVLAKLENGEWKVLYRHCTGDNEECYDRQECVFTESGSPYKYCCCEGNFCNERFTHLPEPG
>sequence_4
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCSVLAKLENGKWEILYRHCTGDNEECYDRQECVSRSSGSPYQYCCCEGNFCNERFTHLPEPG
>sequence_5
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYRAWKRENGKDTIVYSGCTEDNDECYDRQECVSTQKGAKILSCCCEGNFCNERFTHLPEPG
>sequence_6
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYAAYKLENGKKTVILRGCTTDNDECYDRQECVSTEKGAKILSCCCEGNFCNERFTHLPEPG
>sequence_7
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCTVLAKVENGKLIRVYSGCTEDLDECYDRQECVATEKGAKYLSCCCEGNFCNERFTHLPEPG
>sequence_8
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCSVLYKIENGELKVVYRGCTEDLDECYDRQECVGTDRNAPYIFCCCEGNFCNERFTHLPEPG
>sequence_9
TRECIYYNANWELERTNQSGLERCEGEQDKRLHCGVLVKYENGEWTVLYSGCVGDNEECYDRQECVATESGKEYQYCCCEGNFCNERFTHLPEPG
>sequence_10
TRECIYYNANWELER

In [10]:
#submit and monitor job status
response_alignment = requests.post("https://www.ebi.ac.uk/Tools/services/rest/clustalo/run", data={
    "email": "example@example.com",
    "iterations": 0,
    "outfmt": "clustal_num",
    "order": "aligned",
    "sequence": content,
})

job_id = response_alignment.text
job_status = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}")

while status_check():
    job_status = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}")
    time.sleep(3)

RUNNING
RUNNING
RUNNING
FINISHED


In [11]:
#show alignment
show_alignment()

CLUSTAL O(1.2.4) multiple sequence alignment


sequence_11      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCFRAYRIVNGKKEILYSGCVENEDSCY	60
sequence_12      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCFRAYRIENGKVHILYSGCVENEDECY	60
sequence_8       TRECIYYNANWELERTNQSGLERCEGEQDKRLHCSVLYKIENGELKVVYRGCTEDLDECY	60
sequence_29      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCFALYRIENGKTTWLYRGCTTDNDECY	60
sequence_30      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYRAYRIENGEDIVLTQGCTTDNDECY	60
sequence_39      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCVRAYKVENGVTTILYSGCATNLDECY	60
sequence_40      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCIRAYKIENGKTTILYSGCATNLDECY	60
sequence_54      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCMAAAKLINGEWEILYRGCTGDNDECY	60
sequence_52      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCLRAAKLENGKTTLLYSGCTGETDECY	60
sequence_50      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCVVAYKEENGKTTILYTGCTYDTDECY	60
sequence_43      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYVLAKLENGVWTVLYRGCTGENDECY	60
sequence_44      TRECIYYNANWELERTNQSGLERCEGEQDKRLHCYVLAKTENGVT

In [12]:
#create alignment file
alignment = (get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/aln-clustal_num")).text

with open(f'11_typeII_interface_aligned_MSA.aln', 'w+') as file:
    file.writelines(alignment)

In [16]:
#turn MSA file into fasta file using BioPython
input_clustal_file = f"11_typeII_interface_aligned_MSA.aln"
output_fasta_file = f"11_typeII_interface_aligned_fasta.fasta"

convert_clustal_to_fasta(input_clustal_file, output_fasta_file)

In [18]:
#Generate multiple sequence alignment image
msa_file = open(f"11_typeII_interface_aligned_fasta.fasta")
mv = MsaViz(msa_file, wrap_length=60, show_count=True)
mv.savefig(f"11_typeII_interface.png")

In [83]:
from weblogo import read_seq_data, LogoData, LogoOptions, LogoFormat, pdf_formatter, std_color_schemes

# Load sequences from FASTA file
with open("11_typeII_interface.fasta", 'r') as f:
    seqs = read_seq_data(f)

# Generate Logo
data = LogoData.from_seqs(seqs)
options = LogoOptions()
options.title = "My Sequence Logo"
options.color_scheme = std_color_schemes['chemistry']
format_ = LogoFormat(data, options)

# Save the logo to a file
with open("sequence_logo_chemistry_v2.pdf", "wb") as f:
    f.write(pdf_formatter(data, format_))

In [103]:
from PIL import Image, ImageDraw, ImageFont

# Load the sequence logo
img = Image.open("sequence_logo_chemistry_v2.png")

# Get a drawing context
draw = ImageDraw.Draw(img)

# Define the wild type sequence
wild_type_seq = "ATGCGTAGCTAGCTAG"

# Define font and position
font = ImageFont.load_default()
# text_width, text_height = draw.textsize(wild_type_seq, font=font)  # Corrected here
# text_position = (img.width/2 - text_width/2, img.height - text_height - 10)

# Draw the wild type sequence onto the image
draw.text((0,0), wild_type_seq, font=font, fill="black")

# Save the modified image
img.save("sequence_logo_with_wildtype.png")
